# Cruz Hacks Stable Diffusion Prompt Generator and Draw Things Driver

This notebook shows how to generate a list of prompts for a text to image application. This one generates a list of illustrations of a diverse cast of people working at different jobs. It also shows how to use this list of prompts to drive the Draw Things application to actually generate the images.

This notebook shows how to use Beautiful Soup to scrape job descriptions from the 4 corner resources website. This is purely for demo purposes, in order to use this data, you should contact 4cornerresources.com.

Fetches the main webpage into a soup.

In [3]:
import requests
import random
from pprint import pprint
from bs4 import BeautifulSoup
url = "https://www.4cornerresources.com/job-descriptions/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

Scrapes the job descriptions. For each job, creates a dictionary where the first entry is 'job' : '*job*' and the second entry is 'prompt' : 'a simple drawing of a *race* *gender* *job* at work, a colorful cute illustration, well defined'. Then it puts all of these dictionaries into a list.

In [4]:
links = {}
prompts = []
race = ["black", "asian", "white", "hispanic", "indian"]
gender = ["woman", "man"]
for lettersection in (soup.find_all('div', class_='letter-section')):
  for job in lettersection.find_all('a'):
    link = job.get('href')
    if not str(link).startswith('#'):
      jobDescription = job.get_text('a').strip()
      prompts.append({"job" : jobDescription,  "prompt" : "a simple drawing of a " + random.choice(race) +" "  +random.choice(gender)+ " " + jobDescription+ " at work, a colorful cute illustration, well defined"  })
pprint(prompts)

[{'job': 'Accounting Assistant',
  'prompt': 'a indian man Accounting Assistant at work, a colorful cute '
            'illustration, well defined '},
 {'job': 'Accounting Clerk',
  'prompt': 'a indian woman Accounting Clerk at work, a colorful cute '
            'illustration, well defined '},
 {'job': 'Accounting Director',
  'prompt': 'a black man Accounting Director at work, a colorful cute '
            'illustration, well defined '},
 {'job': 'Accounts Payable Clerk',
  'prompt': 'a asian man Accounts Payable Clerk at work, a colorful cute '
            'illustration, well defined '},
 {'job': 'Accounts Payable Specialist',
  'prompt': 'a hispanic woman Accounts Payable Specialist at work, a colorful '
            'cute illustration, well defined '},
 {'job': 'Accounts Receivable Clerk',
  'prompt': 'a white woman Accounts Receivable Clerk at work, a colorful cute '
            'illustration, well defined '},
 {'job': 'Account Executive',
  'prompt': 'a indian woman Account Execu

Finally, puts the dictionary into a json and saves it to a file.

In [6]:
import json
with open('images.json', 'w') as fp:
    json.dump(links, fp)

# and to downlaod from colab, do the following
# from google.colab import files
# files.download('images.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Drive the Draw Things application to generate the images. Goes through the list of prompts and gets the job description and job title. Then it calls the Draw Things web interface to generate the image. A fixed seed is used to get consistent visualization each time the script is run. Then it saves the image as job title.png.

You must have Draw Things app open and the Draw things "HTTP API SERVER" setting enabled.

Draw Things is a Macintosh appliaction for Stable Diffusion text to image generation.
https://drawthings.ai/

In [ ]:
import requests
from PIL import Image
import io
import base64

# Adapted from https://discord.com/channels/1038516303666876436/1196982430989557800/1196982430989557800

# Function to send a request to the stable-diffusion API
def generate_image(prompt):
    url = "http://127.0.0.1:7860/sdapi/v1/txt2img"
    payload = {"prompt": prompt, "steps": 40, "seed": 960515695}
    response = requests.post(url, json=payload)
    return response.json()


# Function to process the response and save the image
def save_image(image_data, file_name):
    image = Image.open(io.BytesIO(base64.b64decode(image_data)))
    try:
        image.save(file_name)
    except Exception as err:
        print(f"couldn't write file name {file_name} err: {err}")


# Function to get the prompts
def get_roles():
    return links

# Main
for i, role in enumerate(get_roles()):
    response_data = generate_image(role["prompt"])
    if "images" in response_data and response_data["images"]:
        save_image(response_data["images"][0], f'{role["job"]}.png')